In [2]:
import quantecon as qe
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import optimize
import pandas as pd
from scipy.optimize import fsolve

#### Problem Set 5: Solving Aiyagari Model

##### Hasan Cetin, Lucas Belmudes

---

### The Problem

The Bellmann equation of the consumer is the following:

$$ T(V_n)(l,k) = \max_{k'} \left \{ \dfrac{(z-k')^{1-α}}{1-α} + \mathbb{E} \left \{ V_n(l',k') | l \right \} \right \}$$
$$\text{s.t.   } 0 \leq k' \leq z$$
$$\text{where   } z = wl + (1+r)k$$
$$w = (1-θ)\dfrac{K^{α}}{L}, \;\; r = θ \dfrac{K^{-α}}{L} $$

- Assume measure 1 of consumers

- We will assume $l$ is an i.i.d. idiosyncratic process centered at 1. (iid for simplicity, idiosyncracy by Aiyagari Model)

- Thus $L = 1$  and $w,r$ depends only on $K^D$: capital demand.

- $K^S$: capital supply is provided by the consumers/households.

- We will use EGM as in HW4 to solve the consumption-savings problem.

#### The algorithmus

1. Start $K^D$ from $K_{grid}$ and calculate prices $w,r$

2. Solve consumption-savings problem (we used EGM for efficiency) and get $V^*$ and corresponding $Γ^*$

3. Start a simulation for N agents starting from some $k_0$ by creating an $N$ x $T$ shock grid and use $Γ^*$ to simulate their path.
     -  NOTE: Use same shocks for each iteration!!!!!
     -  NOTE: Since this shock grid will use too much RAM size (if T = 20000, N = 1000, then size = 20000 x 1000 x 8 byte = 16 x $10^8$ = 1.6 GB), after calculating path, erase this shock grid and use a seed to create the same shock grid again.

4. $K^S = \dfrac{\sum_{i=1}^{N} k_i^T}{N}$

5. If $K^S > K^D$ increase $K^D$ (i.e. go to next value in the $K_{grid}$ for $K^D$)
     - NOTE: Check the following two cases first:
          - If $K^D > K^S$ even for $K^D = K_{grid}[0]$ then decrease your $K_{grid}$ immediately and start again.
          - If $K^D < K^S$ even for $K^D = K_{grid}[max]$ then increase your $K_{grid}[max]$ immediately and start again.
     - NOTE: If $K^D > K^S$ for some in $K_{grid}$ and $K^D < K^S$ for others in $K_{grid}$ but still no convergence, use finer $K_{grid}$

6. Iterate this process until market clearence: $max(|K^S - K^D|) < ϵ$

7. If no convergence, make your $K_{grid}$ finer.



In [46]:
# Define Parameters:

N = 100   # Number of agents for simulation
T = 20000 # Number of periods for simulation
n_k = 500 # Grids for capitals
n_A = 19  # Markov States.
δ = 1     # Full Depreciation.
α = 0.7   # Capital Share.
ρ = 0.95  # Memory of income
σ = 0.2   # Volatility of income.
β = 0.96  # Discont factor.
θ = 1     # Expanding grid coefficient.
error = 10e-6   # Error tolerance.
max_iter = 1000 # Maximum iteration
delta = 10e-4   #To differentiate the value function

np.random.seed(1234) #We are using a seed to generate the same shock grid each time

#Create aux variables for iteration
dist = 1000
iteration = 0

#Discretizing shocks
markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)
Π = markov.P   
A = markov.state_values 

#To be able to simulate the income shock process we will create cumulative prob matrix Π_sim
Π_sim = np.zeros((n_A, n_A))
for i in range(n_A):
    sum = 0
    for j in range(n_A):
        sum += Π[i,j] 
        Π_sim[i,j] = sum

# Maximum sustainable Capital:
K_max = 5
K_min = 0.01

# Use and expanding grid for more accuracy:
K_grid = K_min + (K_max - K_min) * (np.linspace(0, 1, n_k)**θ)

In [56]:
for Kd_index, Kd in enumerate(K_grid):
    #Calculate the prices first:
    w = (1-α)*(Kd**(α))
    r = α * (Kd**(-α))

    #Given Prices, solve the consumption-saving problem:
    #Step 0: Some necessary matrices for EGM.
    V_k = np.zeros((n_A,n_k))
    c_star = np.zeros((n_A,n_k))
    Y_star = np.zeros((n_A,n_k))
    value = np.zeros((n_A,n_k))
    Realvalue = np.zeros((n_A,n_k))

    #Step 1: Creating on grid cash in hands matrix (i.e. z)
    Y_prime = np.zeros((n_A,n_k))
    for i in range(n_A):
        for j in range(n_k):
            Y_prime[i,j] = A[i]*w + K_grid[j]*(1+r)

    #Step 2: Initial guess
    V_0 = np.zeros((n_A,n_k))
    for i in range(n_A):
        for j in range(n_k):
            V_0[i,j] = K_grid[j]**(α) * A[i]

    #Finding the value function
    while dist > error and iteration < max_iter:
        for i in range(n_A):
            #Step 3: Doing the numerical derivative of the value matrix wrt k to get c_star
            V_spline = CubicSpline(K_grid,V_0[i,:])
            c_star[i,:] = ((V_spline(K_grid + delta) - V_spline(K_grid - delta))/ (2*delta))**(-1/2)
        
        #Step 4: Finding Y_star
        Y_star = c_star + K_grid
        
        #Step 5: Finding corresponding values
        Values = -c_star **(-1) + V_0
        
        for i in range(n_A):
            Y_i = Y_star[i,:]
            V_i = Values[i,:]

            #We need to sort Y_star to be able to interpolate
            sort_index = np.argsort(Y_i)
            Y_i_sorted = Y_i[sort_index]
            V_i_sorted = V_i[sort_index]

            #Step 6: Interpolate to get on grid values
            interpolation = CubicSpline(Y_i_sorted, V_i_sorted)
            Realvalue[i,:] = interpolation(Y_prime[i,:])

        #Step 7: Update your value
        V_1 = β * np.dot(Π, Realvalue)

        #Step 8: Check your
        dist = np.max(np.abs(V_1 - V_0))
        iteration += 1

        #Step 7: Update your value
        V_0 = V_1.copy()

        #Checking convergence of the value function
        if iteration > max_iter - 1:
            print("VFI does not converge")
            break

    #Step 8: Find the policy function
    k_policy = np.zeros((n_A,n_k))    #Real k'
    policy_index = np.zeros((n_A,n_k))#To find the index of policies for simulation
    for i in range(n_A):
        for j in range(n_k):
            def f(k, z=A[i], y_star=Y_star[i,j]):
                return z * k**α + (1-δ)*k - y_star
            k_policy[i,j] = fsolve(f,K_grid[0])
    #Step 9: Find the index matrix of the policy function for the simulation
    for i in range(n_A):
        for j in range(n_k):
            Grid = k_policy[i,j] - K_grid #We will use the closest point in K_grid to the real k'
            policy_index[i,j] = np.searchsorted(Grid,0)

    #Step 10: Simulation to find Ks
    #Create income shock simulation, need to use uniform rv to be able to use Π Markov Process
    Shock_sim = np.random.uniform(size=(N,T))
    
    #Create Income simulation (it gives the income shock indexes) and Policy simulation
    Income_sim = np.zeros((N,T+1))
    K_sim = np.zeros((N,T+1))
    
    #Assume each agent starts with mean income and 0 capital
    Income_sim[:,0] = n_A//2
    K_sim[:,0] = 0

    for n in range(N):
        for t in range(T):
            #Finding the income simulation
            new_shock = np.searchsorted(Π_sim[Income_sim[n,t],:], Shock_sim[n,t])
            Income_sim[n,t+1] = new_shock

            #In case np.searchsorted() function messes up
            #It is 0 possibility but if the uniform shock is exactly 1 then function messes up
            if new_shock >= n_A: 
                print("np.searchsorted is messed up!")
                break

            #Given income shock find the policy simulation
            K_sim[n,t+1] = policy_index[Income_sim[n,t],K_sim[n,t]]
  
    #Step 11: Calculate Ks
    Ks = np.sum(K_grid[K_sim[:,-1]])/N

    #Step 12: Check market clearence
    Diff = Ks - Kd 
    if Diff < 0:
        print("Ks < Kd")
        break
    elif Kd_index % 10 == 0:
        print("iteration: ", Kd_index)
        print("Difference: ", Diff)

    # If Market clears finish the program 
    if Diff < error:
        w_eq, r_eq = w, r #Equilibrium Prices
        Kd_eq = Kd        #Equilibrium Capital
        Sim_path_eq = K_sim #Simulation path in equilibrium prices
        break
    
    # To create a space in our beloved RAM, we erase this gigantic simulation grids. 
    del Shock_sim
    del Income_sim
    del K_sim

/var/folders/c_/9slv16s13p50vtr162z5f8th0000gn/T/ipykernel_97745/2295424456.py:32: RuntimeWarning: invalid value encountered in power
  c_star[i,:] = ((V_spline(K_grid + delta) - V_spline(K_grid - delta))/ (2*delta))**(-1/2)


ValueError: `x` must contain only finite values.

Note that `np.searchsorted()` to find income shock is essentially wrong. Why? because let's say the uniform number is 0.68 and it is between to indexes in Π_sim of 0.67 and 0.7. `np.searchsorted()` would give the index of 0.67 but the correct index is of 0.7's because these are cumulative probabilities. But since we are using high N and T and since these type of cases are minimal, we ignore this problem.